# Exercise 02: Complex Exponentials

In this exercise, you will explore the connections between complex exponentials and sinusoids (sines and cosines).

Let's start by installing the necessary packages:

In [ ]:
import sys
!{sys.executable} -m pip install -U pip
!{sys.executable} -m pip install -U numpy matplotlib ipywidgets ipympl

and importing the needed modules:

In [ ]:
%matplotlib widget

import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

The following code creates an interactive plot.  Feel free to try to understand what it's doing.

In [ ]:
def create_complex_exponential_plot():
    cb_pos = widgets.Checkbox(value=False, description='pos: ' + r'$e^{+j\theta}$')
    cb_neg = widgets.Checkbox(value=False, description='neg: ' + r'$e^{-j\theta}$')
    cb_sum = widgets.Checkbox(value=False, description='sum: ' + r'$e^{+j\theta} + e^{-j\theta}$')
    cb_dif = widgets.Checkbox(value=False, description='dif: ' + r'$e^{+j\theta} - e^{-j\theta}$')

    thetas = np.linspace(0.0,+20.0,1000)

    fig = plt.figure()
    fig.canvas.toolbar_visible = False
    fig.canvas.resizable       = False
    fig.canvas.header_visible  = False
    fig.set_size_inches(8, 6)
    
    ax  = fig.add_subplot(1, 1, 1, projection='3d')
    ax.set_xlim(0.0, +20.0)
    ax.set_ylim(-2.0, +2.0)
    ax.set_zlim(-2.0, +2.0)
    ax.set_xlabel(r'$\theta \; [rad]$')
    ax.set_ylabel(r'$real$')
    ax.set_zlabel(r'$imag$')

    pos_real_values = np.cos(thetas)
    pos_imag_values = np.sin(thetas)

    neg_real_values = np.cos(-thetas)
    neg_imag_values = np.sin(-thetas)

    axis_line_x   = ax.plot([ 0, 20], [0, 0], [0, 0], color='k', visible=True)
    axis_line_y_1 = ax.plot([ 0,  0], [0, 2], [0, 0], color='k', visible=True)
    axis_line_y_2 = ax.plot([20, 20], [0, 2], [0, 0], color='k', visible=True)
    axis_line_z_1 = ax.plot([ 0,  0], [0, 0], [0, 2], color='k', visible=True)
    axis_line_z_2 = ax.plot([20, 20], [0, 0], [0, 2], color='k', visible=True)
    
    pos_line, = ax.plot(thetas, pos_real_values, pos_imag_values, color='g', visible=False)
    neg_line, = ax.plot(thetas, neg_real_values, neg_imag_values, color='r', visible=False)
    sum_line, = ax.plot(thetas, pos_real_values + neg_real_values, pos_imag_values + neg_imag_values, color='m', visible=False)
    dif_line, = ax.plot(thetas, pos_real_values - neg_real_values, pos_imag_values - neg_imag_values, color='c', visible=False)

    def update_plot(dummy):
        pos_line.set_visible(cb_pos.value)
        neg_line.set_visible(cb_neg.value)
        sum_line.set_visible(cb_sum.value)
        dif_line.set_visible(cb_dif.value)
        fig.canvas.draw_idle()

    cb_pos.observe(update_plot, names='value')
    cb_neg.observe(update_plot, names='value')
    cb_sum.observe(update_plot, names='value')
    cb_dif.observe(update_plot, names='value')

    display(
        widgets.HBox([
            widgets.VBox([
                cb_pos, cb_neg,
            ]),
            widgets.VBox([
                cb_sum, cb_dif,
            ]),
        ])
    )
create_complex_exponential_plot()

As before, you might want/need to occasionally close all plots to avoid leaking memory:

In [ ]:
plt.close('all')

By clicking-and-dagging,
rotate the plot
so that you're looking down the positive $\theta$ axis toward the origin.
This view is essentially a complex plane
with a horizontal real axis
and a vertical imaginary axis.

Now rotate back to a 3D perspective.

Instead of plotting individual points in the complex plane,
we're going to be plotting curves
that show how points move as $\theta$ changes.

If we slice the diagram at a given value of $\theta$,
we end up with a complex plane.

**MAKE SURE YOU ARE CLEAR ON THIS CONCEPT BEFORE CONTINUING!**

Enable the positive $e^{+j\theta}$ complex exponential.  Study the plot and make sure you understand what you're seeing.

Now enable the negative $e^{-j\theta}$ complex exponential.  Study the plot and make sure you understand what you're seeing, toggling the positive and negative waveforms as desired.

**How do the positive and negative complex exponentials differ?**

YOUR ANSWER HERE

Enable the sum waveform (toggling the others as desired) and study the plot.

**What do you notice about the sum?  What important property is being demonstrated here?**

YOUR ANSWER HERE

Disable the sum waveform, and enable the difference.  Again, study the plot.

**What do you notice about the difference? What important property is being demonstrated here?**

YOUR ANSWER HERE

Now enable only the sum and difference waveforms.

View the plot from the side, and then from the top.

**What do you notice?**

YOUR ANSWER HERE